# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [3]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")
df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,1/6/11,Employed,F,...,0.0,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2/6/11,Employed,F,...,0.0,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,NaN
10908,10908,WA60547,California,11971.977650,No,Premium,College,2/13/11,Employed,F,...,4.0,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A


In [4]:
#Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 
#and have a response of "Yes" to the last marketing campaign.

df.columns=[col.replace(" ","_").lower() for col in df.columns]

In [17]:
fildf=df[df["total_claim_amount"] > 1000]
fildf[fildf["response"] == "Yes"]
fildf.reset_index(drop=True,inplace=True)
fildf

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,30,GX19342,California,8491.465719,No,Premium,High School or Below,1/28/11,Unemployed,F,...,0.0,3,Personal Auto,Personal L3,Offer1,Branch,1193.115879,Four-Door Car,Large,A
1,55,XQ74827,Arizona,19435.629240,No,Extended,High School or Below,2/1/11,Employed,F,...,0.0,3,Special Auto,Special L3,Offer1,Agent,2294.631639,Luxury Car,Medsize,A
2,74,LD75023,Oregon,26346.085820,No,Extended,High School or Below,1/20/11,Employed,M,...,0.0,2,Personal Auto,Personal L2,Offer1,Web,1605.600000,Luxury SUV,Medsize,NaN
3,77,YC54142,Washington,74228.516040,No,Extended,High School or Below,1/26/11,Unemployed,M,...,0.0,2,Personal Auto,Personal L1,Offer1,Branch,1742.400000,Luxury Car,Medsize,NaN
4,99,JY66887,California,16320.767160,No,Extended,High School or Below,2/27/11,Unemployed,F,...,3.0,5,Personal Auto,Personal L2,Offer2,Branch,1778.400000,Luxury Car,Medsize,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,10874,PD52748,NaN,10036.536340,NaN,Extended,High School or Below,2/12/11,Unemployed,F,...,1.0,8,Personal Auto,Personal L3,Offer1,Call Center,1353.289060,SUV,Medsize,NaN
487,10879,FK95213,California,18611.453560,No,Extended,Master,1/16/11,Employed,F,...,0.0,7,Personal Auto,Personal L3,Offer2,Web,1021.654003,NaN,NaN,NaN
488,10900,VN73653,Oregon,6423.739052,No,Basic,College,2/13/11,Unemployed,M,...,0.0,1,Corporate Auto,Corporate L2,Offer3,Agent,1621.560425,Luxury SUV,Medsize,NaN
489,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,1/19/11,Unemployed,F,...,NaN,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A


In [19]:
#Using the original Dataframe, analyze the average total_claim_amount
#by each policy type and gender for customers who have responded "Yes" 
#to the last marketing campaign. Write your conclusions.

pivotdf=df.pivot_table(index=["policy_type","gender"],values=["total_claim_amount"],aggfunc="mean")
pivotdf

total_claim_amount
policy_type    gender                    
Corporate Auto F               397.799287
               M               462.223565
Personal Auto  F               413.239658
               M               459.919476
Special Auto   F               458.139623
               M               420.355202

In [21]:
#Analyze the total number of customers who have policies in each 
#state, and then filter the results to only include states where there are more than 500 customers.
pivotdf1=df.pivot_table(index=["state"],values=["customer"],aggfunc="count")
pivotdf1

,customer
state,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888


In [22]:
filpivotdf1=pivotdf1[pivotdf1["customer"] > 500]

In [23]:
filpivotdf1

,customer
state,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888
